# KNN für die Zalando-Artikel

Ca. 60000 Fotos von Zalando-Artikeln wurden als Graustufenbild mit einer Auflösung von 28x28 bereitgestellt. dabei handelt es sich um Zalando-Waren, die gelabelt sind in 10 Kategorien.

Mit diesen Daten wird hier ein 3-stufiges KNN trainiert.

Das KNN hat also:
- 28x28 = 784 Input-Neuronen
- 10 Output-Neuronen
- sowie eine verdeckte Schicht mit einer per Parameter steuerbaren Anzahl Neuronen

## Import der notwendigen Bibliotheken

In [ ]:
import numpy as np

# scipy.special for the sigmoid function expit()
from scipy.special import expit 

# helper to load data from PNG image files
# glob helps select multiple files using patterns
import glob

# library for plotting arraysimport imageio
import imageio.v2 as imageio

import pandas as pd

# library for plotting
import matplotlib.pyplot

# ensure the plots are inside this notebook, not an external window
%matplotlib inline

## Namen der Kleidungstypen festlegen

In [ ]:
modelle = {0:"T-shirt", 
           1:"Hose", 
           2:"Pullover",
           3:"Kleid",
           4:"Jacke",
           5:"Sandale",
           6:"Hemd",
           7:"Sneaker",
           8:"Tasche",
           9:"Stiefel"
          }

## Python-Klasse für ein neuronales Netz

**3-schichtiges Neuronales Netz**

Parameter:
- Anzahl Neuronen des Input Layer
- Anzahl Neuronen des Hidden Layer
- Anzahl Neuronen des Output-Layer
- Lernrate

In [ ]:
# neural network class definition
class neuralNetwork:
    
    
    # initialise the neural network
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        # set number of nodes in each input, hidden, output layer
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        
        # link weight matrices, wih and who
        # weights inside the arrays are w_i_j, where link is from node i to node j in the next layer
        # w11 w21
        # w12 w22 etc 
        self.wih = np.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes, self.inodes))
        self.who = np.random.normal(0.0, pow(self.hnodes, -0.5), (self.onodes, self.hnodes))

        # learning rate
        self.lr = learningrate
        
        # activation function is the sigmoid function
        self.activation_function = lambda x: expit(x)
        
        pass

    
    # train the neural network
    def train(self, inputs_list, targets_list):
        # convert inputs list to 2d array
        inputs = np.array(inputs_list, ndmin=2).T
        targets = np.array(targets_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = np.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        final_inputs = np.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        # output layer error is the (target - actual)
        output_errors = targets - final_outputs
        # hidden layer error is the output_errors, split by weights, recombined at hidden nodes
        hidden_errors = np.dot(self.who.T, output_errors) 
        
        # update the weights for the links between the hidden and output layers
        self.who += self.lr * np.dot((output_errors * final_outputs * (1.0 - final_outputs)), np.transpose(hidden_outputs))
        
        # update the weights for the links between the input and hidden layers
        self.wih += self.lr * np.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), np.transpose(inputs))
        
        pass

    
    # query the neural network
    def query(self, inputs_list):
        # convert inputs list to 2d array
        inputs = np.array(inputs_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = np.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        final_inputs = np.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs

## Parameter setzen und KNN aufbauen

In [ ]:
# number of input-, hidden- and output-nodes nodes
input_nodes = 784
hidden_nodes = 200
output_nodes = 10

# learning rate
learning_rate = 0.1

# create instance of neural network
knn = neuralNetwork(input_nodes,hidden_nodes,output_nodes, learning_rate)

## Trainingsdaten laden

In [ ]:
# load the mnist training data CSV file into a list
training_image_file = open("data/X_train.csv", 'r')
training_image_list = training_image_file.readlines()
training_image_file.close()

training_typen_file = open("data/y_train.csv", 'r')
training_typen_list = training_typen_file.readlines()
training_typen_file.close()

## KNN trainieren

In [ ]:
# train the neural network

# epochs is the number of times the training data set is used for training
epochs = 10

print ("Bitte warten!")
for e in range(epochs):
    print(e+1, "von", epochs, "Epochen gestartet.... ", end = "")
    # go through all records in the training data set
    for index in range(60000):
        # split the record by the ',' commas
        all_values = training_image_list[index].split(",")
        # scale and shift the inputs
        inputs = (np.asfarray(all_values) / 255.0 * 0.99) + 0.01
        
        # create the target output values (all 0.01, except the desired label which is 0.99)
        targets = np.zeros(output_nodes) + 0.01
        # all_values[0] is the target label for this record
        targets[int(training_typen_list[index])] = 0.99
        knn.train(inputs, targets)
        pass
    print ("done")
    pass
print ("Done")

# Tool zum Testen der Qualität

In [ ]:
def soll_ist (index, image_list, typen_list):
    img_array = np.asfarray(image_list[index].split(",")).astype(int)
    img_data  = 255.0 - img_array
    img_data  = img_array
    img_data = (img_data / 255.0 * 0.99) + 0.01
    inputs = img_data

    # query the network
    outputs = knn.query(inputs)
    label = np.argmax(outputs)
    correct_label = int(typen_list[index])
    return (correct_label, label)

In [ ]:
soll_ist(42,training_image_list, training_typen_list)

In [ ]:
def soll_ist_all(image_list, typen_list):
    m = np.zeros((10, 10), dtype = int)
    anzahl = len(image_list)
    for index in range (anzahl):
        s, i = soll_ist (index, image_list, typen_list)
        m [s][i] += 1
    return m

In [ ]:
sia_training = soll_ist_all (training_image_list, training_typen_list)

## Güte des KNN mit den Trainingsdaten(!) testen

In [ ]:
def test_all_traindata():
    korrekte = 0
    anzahl = len(training_image_list)
    falsch = {x: 0 for x in modelle.values()}
    wrongIndizes = []
    
    for index in range (anzahl):
        
        img_array = np.asfarray(training_image_list[index].split(",")).astype(int)
        img_data  = img_array
        img_data = (img_data / 255.0 * 0.99) + 0.01

        # data is remaining values
        inputs = img_data

        # query the network
        outputs = knn.query(inputs)
        
        # the index of the highest value corresponds to the label
        label = np.argmax(outputs)
        correct_label = int(training_typen_list[index])
 
        if (label == correct_label):
            korrekte += 1
        else:
            zeile = ""
            zeile += f'Nr. {index:5} Netz sagt: {label:2} Soll: {correct_label:2}'
            falsch[modelle[correct_label]] += 1
            wrongIndizes.append(index)
    print ("Falsch erkannt:")
    for f in falsch:
        print (f + ": " + str(falsch[f]))
    print ("Von den", anzahl, "Testdaten sind", korrekte, "korrekt", anzahl - korrekte, "falsch zugeordnet")
    # print(wrongIndizes)
    
test_all_traindata()


In [ ]:
df = pd.DataFrame(data=sia_training)
df = df.rename(modelle, axis='columns')
df = df.rename(modelle, axis='rows')
df["Fehler"] = ""

for i in range (10):
    f = 0;
    for j in range (10):
        if i != j:
            inhalt = int (df[modelle[j]][modelle[i]])
            f += int (inhalt)
    df["Fehler"][modelle[i]] = f

display (df)

## Testdaten laden

In [ ]:
# load the mnist training data CSV file into a list
test_image_file = open("data/X_test.csv", 'r')
test_image_list = test_image_file.readlines()
test_image_file.close()

test_typen_file = open("data/y_test.csv", 'r')
test_typen_list = test_typen_file.readlines()
test_typen_file.close()

## Funktion zum Testen eines Test-Datensatzes

In [ ]:
def testen (index):
    #img_array = imageio.imread(image_file_name, as_gray=True)
    img_array = np.asfarray(test_image_list[index].split(",")).astype(int)
    # reshape from 28x28 to list of 784 values, invert values
    #img_data  = 255.0 - img_array
    img_data  = img_array
    # then scale data to range from 0.01 to 1.0
    img_data = (img_data / 255.0 * 0.99) + 0.01
    #print(numpy.min(img_data))
    #print(numpy.max(img_data))
        
    # plot image
    matplotlib.pyplot.imshow(img_data.reshape(28,28), cmap='Greys', interpolation='None')

    # data is remaining values
    inputs = img_data

    # query the network
    outputs = knn.query(inputs)
    for i in range (output_nodes):
        print (f'Güte der Erkennung {outputs[i][0]:1.5f} : {modelle[i]}')
        
    # the index of the highest value corresponds to the label
    label = numpy.argmax(outputs)
    correct_label = int(test_typen_list[index])
    print("should be", correct_label, "(", modelle[correct_label],")")
    print("network says ", label, "(", modelle[label],")")


    # append correct or incorrect to list
    if (label == correct_label):
        print ("match!")
    else:
        print ("no match!")


## Testdaten-Güte

In [ ]:
sia_testdaten = soll_ist_all (test_image_list, test_typen_list)

In der folgenden Tabelle sind die Fehler aufgelistet.

- Steht z.B. in der "Pullover"-Zeile der Wert 17 in der "Tasche"-Spalte, wurden also 17 Pullover fälschlicherweise als Tasche erkannt.

In [ ]:
df = pd.DataFrame(data=sia_testdaten)
df = df.rename(modelle, axis='columns')
df = df.rename(modelle, axis='rows')
df["Fehler"] = ""

for i in range (10):
    f = 0;
    for j in range (10):
        if i != j:
            inhalt = int (df[modelle[j]][modelle[i]])
            f += int (inhalt)
    df["Fehler"][modelle[i]] = f

display (df)

## Ein Testdatensatz wird getestet

In [ ]:
testen (42)

## Funktion, um alle Testdaten zu testen

In [ ]:
def test_all():
    korrekte = 0
    log = ""
    anzahl = len(test_image_list)
    falsch = {x: 0 for x in modelle.values()}
    wrongIndizes = []
    
    for index in range (anzahl):
        
        img_array = np.asfarray(test_image_list[index].split(",")).astype(int)
        img_data  = img_array
        img_data = (img_data / 255.0 * 0.99) + 0.01

        # data is remaining values
        inputs = img_data

        # query the network
        outputs = knn.query(inputs)
        
        # the index of the highest value corresponds to the label
        label = np.argmax(outputs)
        correct_label = int(test_typen_list[index])
 
        if (label == correct_label):
            korrekte += 1
        else:
            zeile = ""
            zeile += f'Nr. {index:5} Netz sagt: {label:2} Soll: {correct_label:2}'
            log += zeile + "\n"
            falsch[modelle[correct_label]] += 1
            wrongIndizes.append(index)
    #print (log)
    print ("Falsch erkannt:")
    for f in falsch:
        print (f + ": " + str(falsch[f]))
    print ("Von den", anzahl, "Testdaten sind", korrekte, "korrekt", anzahl - korrekte, "falsch zugeordnet")
    # print(wrongIndizes)


In [ ]:
test_all()